# DA4 - Module 5 - Data Warehouses in the Cloud
## Notebook 2: Building the Data Warehouse in Python

---


---

## The SSMS to Python Mapping

Everything you did yesterday in SSMS has a Python equivalent:

| Yesterday in SSMS | Today in Python (pandas) |
|---|---|
| `CREATE TABLE` | `pd.DataFrame` |
| `SELECT DISTINCT` | `.drop_duplicates()` |
| `IDENTITY(1,1)` surrogate key | `range(1, len+1)` |
| `INSERT INTO ... SELECT` | DataFrame assignment |
| `JOIN` | `pd.merge()` |
| `COUNT(*)` validation | `len()` or `.shape[0]` |

Same concepts. Same data. Different tool.

---
## Step 0: Load the Staging Table



In [ ]:
import pandas as pd

GCS_URL = "https://storage.googleapis.com/ingwane-da4/Superstore.csv"

# Note: encoding='latin1' handles special characters in the data
staging = pd.read_csv(GCS_URL, encoding='latin1')

# Convert date columns to proper datetime types
staging['Order Date'] = pd.to_datetime(staging['Order Date'], dayfirst=True)
staging['Ship Date']  = pd.to_datetime(staging['Ship Date'],  dayfirst=True)

print(f"Staging table loaded: {len(staging):,} rows, {len(staging.columns)} columns")

---
## Step 1: dimCustomer



In [ ]:
# DEMO: Build dimCustomer
# Step 1 - Select only the customer columns and remove duplicates

dimCustomer = (
    staging[['Customer ID', 'Customer Name', 'Segment']]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Step 2 - Add a surrogate key (Customer_SK)
dimCustomer.insert(0, 'Customer_SK', range(1, len(dimCustomer) + 1))

# Step 3 - Rename columns to match DW naming convention
dimCustomer.columns = ['Customer_SK', 'CustomerID', 'CustomerName', 'Segment']

print(f"dimCustomer: {len(dimCustomer):,} rows")
dimCustomer.head()

In [ ]:
# Validate - does the count make sense?
# We expect one row per unique customer

print(f"Unique Customer IDs in staging : {staging['Customer ID'].nunique():,}")
print(f"Rows in dimCustomer            : {len(dimCustomer):,}")
print(f"Match: {staging['Customer ID'].nunique() == len(dimCustomer)}")

---
## Step 2: dimProduct



In [ ]:
# DEMO: Build dimProduct

dimProduct = (
    staging[['Product ID', 'Product Name', 'Category', 'Sub-Category']]
    .drop_duplicates(subset=['Product ID'])
    .reset_index(drop=True)
)

dimProduct.insert(0, 'Product_SK', range(1, len(dimProduct) + 1))

dimProduct.columns = ['Product_SK', 'ProductID', 'ProductName', 'Category', 'SubCategory']

print(f"dimProduct: {len(dimProduct):,} rows")
dimProduct.head()

In [ ]:
# Validate dimProduct

print(f"Unique Product IDs in staging : {staging['Product ID'].nunique():,}")
print(f"Rows in dimProduct            : {len(dimProduct):,}")
print(f"Match: {staging['Product ID'].nunique() == len(dimProduct)}")

---
## 📝 TASK 1 - Build dimGeography


Using the same pattern as dimCustomer and dimProduct, build **dimGeography**.

It should contain: `Country`, `City`, `State`, `Postal Code`, `Region`

The surrogate key should be called `Geog_SK`.

**Hint:** Use `drop_duplicates(subset=['City', 'State'])` - can you think why?

How many rows do you expect?

In [ ]:
# TASK 1 - Build dimGeography

# Your code here


### ✅ TASK 1 SOLUTION



---
## Step 3: dimDate



In [ ]:
# DEMO: Build dimDate
# dimDate is a special dimension - we generate it rather than extract it from staging
# It covers every single day in the date range of our data

# Step 1 - Find the date range from staging
min_date = staging['Order Date'].min()
max_date = staging['Order Date'].max()
print(f"Date range: {min_date.date()} to {max_date.date()}")

In [ ]:
# Step 2 - Generate every date in the range
# Compare this to the WHILE loop approach in T-SQL - much cleaner!

all_dates = pd.date_range(start=min_date, end=max_date, freq='D')
print(f"Total dates generated: {len(all_dates):,}")

In [ ]:
# Step 3 - Build the full dimDate table with useful date attributes

dimDate = pd.DataFrame({'DateValue': all_dates})

dimDate['Day']         = dimDate['DateValue'].dt.day
dimDate['DayOfWeek']   = dimDate['DateValue'].dt.day_name()
dimDate['Week']        = dimDate['DateValue'].dt.isocalendar().week.astype(int)
dimDate['Month']       = dimDate['DateValue'].dt.month
dimDate['MonthName']   = dimDate['DateValue'].dt.month_name()
dimDate['Quarter']     = dimDate['DateValue'].dt.quarter
dimDate['Year']        = dimDate['DateValue'].dt.year

# Add surrogate key
dimDate.insert(0, 'Date_SK', range(1, len(dimDate) + 1))

print(f"dimDate: {len(dimDate):,} rows")
dimDate.head(10)

---
## Step 4: Fact Table - factOrderItem



In [ ]:
# DEMO: Build factOrderItem
# Step 1 - Start with staging and merge in dimCustomer to get the surrogate key

fact = staging.merge(
    dimCustomer[['CustomerID', 'Customer_SK']],
    left_on='Customer ID',
    right_on='CustomerID',
    how='left'
)

print(f"After customer merge: {len(fact):,} rows")

In [ ]:
# Step 2 - Merge in dimProduct

fact = fact.merge(
    dimProduct[['ProductID', 'Product_SK']],
    left_on='Product ID',
    right_on='ProductID',
    how='left'
)

print(f"After product merge: {len(fact):,} rows")

In [ ]:
# Step 3 - Merge in dimGeography

fact = fact.merge(
    dimGeography[['City', 'State', 'Geog_SK']],
    left_on=['City', 'State'],
    right_on=['City', 'State'],
    how='left'
)

print(f"After geography merge: {len(fact):,} rows")

In [ ]:
# Step 4 - Merge in dimDate for Order Date

fact = fact.merge(
    dimDate[['DateValue', 'Date_SK']].rename(columns={'Date_SK': 'OrderDate_SK'}),
    left_on='Order Date',
    right_on='DateValue',
    how='left'
)

print(f"After order date merge: {len(fact):,} rows")

In [ ]:
# Step 5 - Merge in dimDate for Ship Date

fact = fact.merge(
    dimDate[['DateValue', 'Date_SK']].rename(columns={'Date_SK': 'ShipDate_SK'}),
    left_on='Ship Date',
    right_on='DateValue',
    how='left'
)

print(f"After ship date merge: {len(fact):,} rows")

In [ ]:
# Step 6 - Keep only the columns we need in the fact table
# Surrogate keys + measurable facts

factOrderItem = fact[[
    'Row ID',
    'Order ID',
    'Customer_SK',
    'Product_SK',
    'Geog_SK',
    'OrderDate_SK',
    'ShipDate_SK',
    'Sales',
    'Quantity',
    'Discount',
    'Profit'
]].copy()

print(f"factOrderItem: {len(factOrderItem):,} rows, {len(factOrderItem.columns)} columns")
factOrderItem.head()

---
## Step 5: Validation



In [ ]:
# Validation - row counts
print("=== ROW COUNT VALIDATION ===")
print(f"Staging rows     : {len(staging):,}")
print(f"factOrderItem rows: {len(factOrderItem):,}")
print(f"Match            : {len(staging) == len(factOrderItem)}")
print()
print("=== DIMENSION TABLE SIZES ===")
print(f"dimCustomer  : {len(dimCustomer):,} rows")
print(f"dimProduct   : {len(dimProduct):,} rows")
print(f"dimGeography : {len(dimGeography):,} rows")
print(f"dimDate      : {len(dimDate):,} rows")

In [ ]:
# Check for nulls in surrogate key columns
# A null here means a merge (join) failed - something did not match

sk_columns = ['Customer_SK', 'Product_SK', 'Geog_SK', 'OrderDate_SK', 'ShipDate_SK']

print("=== SURROGATE KEY NULL CHECK ===")
for col in sk_columns:
    nulls = factOrderItem[col].isnull().sum()
    status = "✅ OK" if nulls == 0 else f"⚠️  {nulls} nulls!"
    print(f"{col:20} : {status}")

In [ ]:
# Validate totals - Sales total should match staging

print("=== TOTAL SALES VALIDATION ===")
print(f"Staging Sales total      : £{staging['Sales'].sum():,.2f}")
print(f"factOrderItem Sales total: £{factOrderItem['Sales'].sum():,.2f}")
print(f"Match: {round(staging['Sales'].sum(), 2) == round(factOrderItem['Sales'].sum(), 2)}")

---
## 📝 TASK 2 - Validate Your Data Warehouse


Using the dimension tables and fact table you have built, answer the following:

1. How many rows are in the fact table? Does it match staging?
2. What is the total **Quantity** in the fact table? Does it match staging?
3. What is the total **Profit** in the fact table? Does it match staging?
4. Which **year** has the highest total Sales? (Hint: merge factOrderItem with dimDate)

In [ ]:
# TASK 2 - Question 1: Row count check



In [ ]:
# TASK 2 - Question 2: Total Quantity check



In [ ]:
# TASK 2 - Question 3: Total Profit check



In [ ]:
# TASK 2 - Question 4: Which year has highest total Sales?



### ✅ TASK 2 SOLUTIONS



---
## 🚀 STRETCH TASK - dimShipMode


Build a **dimShipMode** dimension table from the staging data.

- It should contain: `Ship Mode`
- The surrogate key should be called `ShipMode_SK`
- How many rows do you expect?
- Once built, can you add `ShipMode_SK` to the fact table?

In [ ]:
# STRETCH - Build dimShipMode



---
## 🍽️ LUNCH


---

## 🔁 Recap - What have we built?

| Table | Type | Rows |
|-------|------|------|
| staging | Raw data | 9,994 |
| dimCustomer | Dimension | 794 |
| dimProduct | Dimension | 1,862 |
| dimGeography | Dimension | 631 |
| dimDate | Dimension | ~1,461 |
| factOrderItem | Fact | 9,994 |

**Next: Notebook 3 - Loading into BigQuery**